# Text Classification:

## Data
<pre>
1. we have total of 20 types of documents(Text files) and total 18828 documents(text files).
2. You can download data from this <a href='https://drive.google.com/open?id=1rxD15nyeIPIAZ-J2VYPrDRZI66-TBWvM'>link</a>, in that you will get documents.rar folder. <br>If you unzip that, you will get total of 18828 documnets. document name is defined as'ClassLabel_DocumentNumberInThatLabel'. 
so from document name, you can extract the label for that document.
4. Now our problem is to classify all the documents into any one of the class.
5. Below we provided count plot of all the labels in our data. 
</pre>

In [1]:
### count plot of all the class labels. 

In [2]:
import nltk
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
import regex as re
import os
import regex as re
from bs4 import BeautifulSoup 
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input,Dense,Conv1D,Flatten,Embedding,MaxPool1D,concatenate,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard,EarlyStopping
from tensorflow.keras.optimizers import Adam

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\scorpions3394\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\scorpions3394\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\scorpions3394\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\scorpions3394\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


#### sample document
<pre>
<font color='blue'>
Subject: A word of advice
From: jcopelan@nyx.cs.du.edu (The One and Only)

In article < 65882@mimsy.umd.edu > mangoe@cs.umd.edu (Charley Wingate) writes:
>
>I've said 100 times that there is no "alternative" that should think you
>might have caught on by now.  And there is no "alternative", but the point
>is, "rationality" isn't an alternative either.  The problems of metaphysical
>and religious knowledge are unsolvable-- or I should say, humans cannot
>solve them.

How does that saying go: Those who say it can't be done shouldn't interrupt
those who are doing it.

Jim
--
Have you washed your brain today?
</font>
</pre>

### Preprocessing:

In [3]:
# 1. FInd all emails in documents and make a list 

import re
def email(doc):
  final_mail = ""
  match = re.findall(r'[\w.+-]+@[\w-]+\.[\w.-]+', doc)
  final_words = []
  for i in match:
    final = i.split('@')[1]
    words = final.split('.')
    for i in words:
      if len(i)>2 and i != 'com':
        final_words.append(i)
  for i in final_words:
      final_mail += " "+i
  return final_mail

In [4]:
# 2. Replace all emails with space
def replace_email(doc):
  rep = re.sub(r'[\w.+-]+@[\w-]+\.[\w.-]+', ' ', doc)
  return rep

In [5]:
# 3. Get subject of the text i.e. get the total lines where "Subject:" occur and remove the word which are before the ":" remove the newlines, tabs, punctuations, any special chars.

def find_subject(doc):
  if(re.findall(r'Subject:.*',doc)):
    subject = re.findall(r'Subject:.*',doc)
    subject = subject[0].split(":")[-1]
    subject = re.sub('[^A-Za-z0-9]+',' ',subject)
    subject = subject.lower()
  return subject

In [6]:
# 4.After you store it in the list, Replace those sentances in original text by space.

def rep_subject(doc):
  rep_subject = re.sub(r'Subject:.*',' ',doc)
  
  return rep_subject


In [7]:
# 5.Delete all the sentances where sentence starts with "Write to:" or "From:".

def rep_from(doc):
  rep_writeto = re.sub(r'Write to:.*',' ',doc)
  rep_from = re.sub(r'From:.*',' ',doc)
  return rep_from

In [8]:
# 6. Delete all the tags like "< anyword >"

from bs4 import BeautifulSoup 

def remove_tag(doc):
  soup = BeautifulSoup(doc,'lxml')
  text = soup.get_text()
  return text


In [9]:
#7. Delete all the data which are present in the brackets.

def rep_bracket(doc):
  string = re.sub("\(.*?\)","",doc)
  return string

In [10]:
#8. Remove all the newlines('\n'), tabs('\t'), "-", "\".

def rep_new_line(doc):
  string = re.sub(r"\s+"," ",doc)
  string = re.sub(r"[/]",".",string)
  return string

In [11]:
#9. Remove all the words which ends with ":".

def rep_word(doc):
  string = re.sub(r"[a-zA-Z]+:"," ",doc)
  return string

In [12]:
#10. Decontractions, replace words like below to full words.

def decontractions(doc):

    string = re.sub(r"can\'t","can not",doc)
    string = re.sub(r"\'s","is",string)
    string = re.sub(r"i\'ve'","i have",string)
    string = re.sub(r"i\'m","i am",string)
    string = re.sub(r"you\'re","you are",string)
    string = re.sub(r"i\'ll","i will",string)

    return string

In [13]:
# 11. Do chunking on the text you have after above preprocessing. 

from nltk import ne_chunk, pos_tag, word_tokenize
def chunking(doc):
    chunks = ne_chunk(pos_tag(word_tokenize(doc)),binary=True)
        
    for item in chunks:
      if type(item)==Tree:
        if item.label()=='PERSON':
          for j,k in item.leaves():
            text = re.sub(r'\b{}\b'.format(j)," ",doc,count=1)
                
        else:
          string = ""
          chunked_string = ""
            
          for j,k in item.leaves():
            string += j + " "
            chunked_string += j + "_"
                        
                
          string = string.strip()
          chunked_string = chunked_string[:-1]
                    
          text = re.sub(r"\b{}\b".format(string),chunked_string,doc,count=1)
                    
    text = re.sub(r"\s+"," ",doc)
    return text

In [14]:
# 13. Replace all the digits with space i.e delete all the digits. 
def rep_digit(doc):
  string = re.sub(r"[0-9]","",doc)
  return string

In [15]:
#14.replace word with _word or word_
def rep_word_(doc):
  string = re.sub(r"(_?)([A-Za-z0-9])(_?)",r'\2',doc)
  return string


In [16]:
#15.We also observed some words like  "OneLetter_word"- eg: d_berlin,

def OneLetter_word(doc):
  string = re.sub(r"([A-Za-z]{1,2})(_)(A-Za-z)","\g<3>",doc)
  return string

In [17]:
#16. Convert all the words into lower case and lowe case and remove the words which are greater than or equal to 15 or less than or equal to 2.

def lower_case(doc):
  string =doc.lower()
  string = ' '.join([w for w in string.split() if len(w)>2 and len(w)<15])

  return string

In [18]:
# 17. replace all the words except "A-Za-z_" with space. 

def final_word(doc):
  string = re.sub(r'[^A-Za-z_]'," ",doc)

  return string

In [19]:
# finally consolidate all function to pre processing a document

def preprocessing(filename):

  class_ = filename.split('_')[0]           # finding a class label from filename

  fhand = open('documents/'+filename, "r")
  final_doc = fhand.read()                          # open a document using filename and path

  emails = email(final_doc)      #------step 1 finding emails

  subject = find_subject(final_doc)           #------------step 3 finding subject for each docuement

  text = replace_email(final_doc) #-------step2

  text = rep_subject(text)  #-------step4

  text = rep_from(text)  #-------step5

  text = remove_tag(text)  #-------step6

  text = rep_bracket(text)  #-------step7

  text = rep_new_line(text)  #-------step8

  text = rep_word(text)  #-------step9

  text = decontractions(text)  #-------step10

  text = chunking(text)  #-------step11

  text = rep_digit(text)  #-------step13

  text = rep_word_(text)  #-------step14

  text = lower_case(text)  #-------step16

  text = final_word(text)  #-------step17

  return [class_,emails,subject,text]



In [20]:
# lets check it is working fine
lst = preprocessing('alt.atheism_51125.txt')

In [21]:
lst

['alt.atheism',
 ' cco caltech edu solntze wpd sgi',
 ' political atheists ',
 '       there good deal more confusion here  you started off with the  assertion that there was some  objective  morality  and you admit  here  you finished with recursive definition  murder   objectively  immoral  but eactly what murder and what not itself  requires appeal morality  yes   now you have switch targets little  but only little  now you are  asking what the  goal   what you mean  goal    are you  suggesting that there some  objective   goal  out there somewhere   and form our morals achieve it  well  for example  the goal  natural  morality the survival and propogation the species  another example moral system presented within the declaration independence  which states that should guaranteed life liberty and the pursuit happiness  you see  have moral system  must define the purpose the system  that is  shall moral unto what end    murder certainly violation the golden rule  and  thought had   de

In [22]:
# preprocess each document in folder

row = []
for f in os.listdir('documents'):
    
    lst = preprocessing(f)
    row.append(lst)

In [23]:
# we have created a list 
row[4]

['alt.atheism',
 ' cco caltech edu jyusenkyou jhu edu',
 ' keith schneider stealth poster ',
 '  but  you were discuss the merits racism  its psycholgical   benefits  you would well have experienced personally   when you speak  experiencing religion  you mean someone should believe religion  thatis right  and this pretty impossible  right  would ideal could believe for while  just try out religion  and only then determine which course thought suits best  but again  this not possible  not that religion warrants belief  but the belief carries with some psychological benefits  there are also some psychological burdens  too   when you speak  experiencing racism   you mean that someone should  believe racism  that they should have racist things done them  for  parallelism  the former must what you meant  but seems odd  usage the phrase  well  there were some psychological other benefits gained from racism  they could only fully understood judged persons actually  believing  racism  course  

In [24]:
# creating a dataframe using pandas dataframe
import pandas as pd
final_data = pd.DataFrame(row,columns = ["class","email","subject","preprocessed_text"])

In [25]:
final_data.head(2)

,class,email,subject,preprocessed_text
0,alt.atheism,cco caltech edu solntze wpd sgi,political atheists,there good deal more confusion here yo...
1,alt.atheism,cco caltech edu CWRU edu,pompous ass,then why people keep asking the same questio...


In [26]:
final_data.to_pickle("./preprocessed_data")

In [27]:
final_data.shape

(18820, 4)

### Training The models to Classify: 


In [28]:
final_data.head(2)

,class,email,subject,preprocessed_text
0,alt.atheism,cco caltech edu solntze wpd sgi,political atheists,there good deal more confusion here yo...
1,alt.atheism,cco caltech edu CWRU edu,pompous ass,then why people keep asking the same questio...


In [29]:
# creating column with consolidated data
final_data['total_data'] = final_data[['email','subject','preprocessed_text']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [30]:
# checking if it is fine
final_data.head(2)

,class,email,subject,preprocessed_text,total_data
0,alt.atheism,cco caltech edu solntze wpd sgi,political atheists,there good deal more confusion here yo...,cco caltech edu solntze wpd sgi political at...
1,alt.atheism,cco caltech edu CWRU edu,pompous ass,then why people keep asking the same questio...,cco caltech edu CWRU edu pompous ass then ...


In [31]:
# lets find X and Y values
final_model_data = final_data[['class','total_data']]
x = final_data['total_data']
y = final_data['class']

In [32]:
final_model_data.shape

(18820, 2)

In [35]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
## encoding lables  Encode target labels with value between 0 and n_classes-1.
encoder = LabelEncoder()
encoder.fit(y)
y_encoder = encoder.transform(y)

#np_utils.to_categorical is used to convert array of labeled data(from 0 to nb_classes - 1) to one-hot vector.
y = np_utils.to_categorical(y_encoder)

In [36]:
# split train and test data in 75% and 25 %

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,stratify=y)

In [37]:
# adding a tokenizer 
tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(x_train)
x_train  = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

In [38]:
x_train = pad_sequences(x_train,maxlen=5000,padding="post")
x_test= pad_sequences(x_test,maxlen=5000,padding='post')

In [39]:
len(x_train[6])

5000

In [40]:
embedding_dict = {}
pretrain =  open("glove.6B.50d.txt",encoding="utf8") 
for i in pretrain:
    value = i.split(" ")
    word = value[0]
    vector = np.asarray(value[1:])
    embedding_dict[word] = vector
pretrain.close()

In [41]:
len(tokenizer.index_word)

80109

In [42]:
# Converting embedding word dictionary to embedding matrix 

emb_matrix = np.zeros((len(tokenizer.index_word)+1,50))

for word,i in tokenizer.word_index.items():
    emb_word = embedding_dict.get(word)

    if emb_word is not None:
      emb_matrix[i]=emb_word

In [43]:
emb_matrix.shape

(80110, 50)

In [44]:
## Embedding layer 
embedding_layer = Embedding(len(tokenizer.word_index)+1, 50, embeddings_initializer=tf.keras.initializers.Constant(emb_matrix))

In [45]:
layer_0 = Input(shape=(5000))

embed_layer = embedding_layer(layer_0)

l_1 = Conv1D(16,4,activation="relu",kernel_initializer =tf.keras.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2())(embed_layer)

l_2 = Conv1D(16,4,activation="relu",kernel_initializer =tf.keras.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2())(embed_layer)

l_3 = Conv1D(16,4,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2())(embed_layer)

second_layer = concatenate([l_1,l_2,l_3])

max_pool_1 = MaxPool1D(3)(second_layer)

l_4 = Conv1D(16,3,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2())(max_pool_1)

l_5 = Conv1D(16,3,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2())(max_pool_1)

l_6 =  Conv1D(16,3,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2())(max_pool_1)

third_layer = concatenate([l_4,l_5,l_6])

max_pool_2 = MaxPool1D(3)(third_layer)

fourth_layer = Conv1D(16,3,activation='relu',
                      kernel_initializer = tf.keras.initializers.he_normal(seed=42),kernel_regularizer=tf.keras.regularizers.l2())(max_pool_2)

flatten = Flatten()(fourth_layer)

dropout_layer = Dropout(0.2)(flatten)

dense_layer = Dense(32,activation="relu",kernel_initializer = tf.keras.initializers.he_normal())(dropout_layer)
              
output_layer = Dense(20,activation="softmax",kernel_initializer= tf.keras.initializers.glorot_normal())(dense_layer)
              

cnn_model =Model(inputs=layer_0,outputs=output_layer) 

In [46]:
cnn_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 5000)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 5000, 50)     4005500     ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 4997, 16)     3216        ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 4997, 16)     3216        ['embedding[0][0]']              
                                                                                              

In [48]:
%load_ext tensorboard

In [49]:
from tensorflow_addons.metrics import F1Score

checkpoint = ModelCheckpoint(filepath='best_model_1.h5',verbose=1,monitor='val_accuracy',
                           mode='auto',save_best_only=True)

## Tensorboard
log_dir = "logs"
tensorboard = TensorBoard(log_dir=log_dir,histogram_freq=1,write_graph=True)

## compile model 
cnn_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy',F1Score(average='micro',num_classes=20)])

## Callback for earlystopping 
early_stop = EarlyStopping(monitor="val_accuracy",mode='max',patience=2,verbose=1)

## Trainning 


cnn_model.fit(x_train,y_train,epochs=10,verbose=2,validation_data=(x_test,y_test),batch_size =64,callbacks=[checkpoint,tensorboard,early_stop]) 

Epoch 1/10

Epoch 1: val_accuracy improved from -inf to 0.22359, saving model to best_model_1.h5
221/221 - 185s - loss: 3.5135 - accuracy: 0.1246 - f1_score: 0.1246 - val_loss: 2.5800 - val_accuracy: 0.2236 - val_f1_score: 0.2236 - 185s/epoch - 837ms/step
Epoch 2/10

Epoch 2: val_accuracy improved from 0.22359 to 0.45845, saving model to best_model_1.h5
221/221 - 179s - loss: 2.1909 - accuracy: 0.3365 - f1_score: 0.3365 - val_loss: 1.8420 - val_accuracy: 0.4584 - val_f1_score: 0.4584 - 179s/epoch - 811ms/step
Epoch 3/10

Epoch 3: val_accuracy improved from 0.45845 to 0.55622, saving model to best_model_1.h5
221/221 - 179s - loss: 1.5833 - accuracy: 0.5408 - f1_score: 0.5408 - val_loss: 1.5347 - val_accuracy: 0.5562 - val_f1_score: 0.5562 - 179s/epoch - 809ms/step
Epoch 4/10

Epoch 4: val_accuracy improved from 0.55622 to 0.62869, saving model to best_model_1.h5
221/221 - 177s - loss: 1.2236 - accuracy: 0.6673 - f1_score: 0.6673 - val_loss: 1.3654 - val_accuracy: 0.6287 - val_f1_score: 

In [54]:
tf.keras.utils.plot_model(cnn_model,to_file = 'model1.png',show_shapes=True,show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


## Model 2

In [55]:
# retrain data with model 2
# train and test split again

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,stratify=y)

In [56]:
# using tokenizer with char level true
# this use character as tokens

tok_char = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',char_level= True,oov_token='UNK')
tok_char.fit_on_texts(x_train)

In [57]:
## Tokenize train and test data using tokenizer
x_train = tok_char.texts_to_sequences(x_train)
x_test = tok_char.texts_to_sequences(x_test)

In [58]:
# padding sequence to same size

x_train = pad_sequences(x_train,maxlen=5000,padding="post")
x_test = pad_sequences(x_test,maxlen=5000,padding="post")

In [59]:
## Make a embedding matrix 

size = len(tok_char.word_index)+1

matrix_char = np.zeros((size,size))

for i,j in tok_char.word_index.items():
  matrix_char[j][j]=1

In [60]:
# creating embedding layer with characters

embedding_layer_char = Embedding(len(tok_char.word_index)+1,41, embeddings_initializer=tf.keras.initializers.Constant(matrix_char),input_length=5000,trainable=False)

In [61]:
# model architecture

layer_0 = Input(shape=(5000))

embed = embedding_layer_char(layer_0)


l_1 = Conv1D(64,3,activation="relu",kernel_initializer =tf.keras.initializers.he_normal(seed=42),kernel_regularizer=tf.keras.regularizers.l1())(embed)

l_2 = Conv1D(64,3,activation="relu",kernel_initializer =tf.keras.initializers.he_normal(seed=42),kernel_regularizer=tf.keras.regularizers.l1())(l_1)


max_pool_1 = MaxPool1D(5)(l_2)

l_3 = Conv1D(64,3,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(seed=42),kernel_regularizer=tf.keras.regularizers.l1())(max_pool_1)

l_4 = Conv1D(64,3,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(seed=42),kernel_regularizer=tf.keras.regularizers.l1())(l_3)


max_pool_2 = MaxPool1D(5)(l_4)

l_5 = Conv1D(64,3,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(seed=42),kernel_regularizer=tf.keras.regularizers.l1())(max_pool_2)


max_pool_3 = MaxPool1D(5)(l_5)

flatten = Flatten()(max_pool_3)

dropout_layer = Dropout(0.5)(flatten)

dense_layer1 = Dense(128,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(seed=42))(dropout_layer)


              
output_layer = Dense(20,activation="softmax",kernel_initializer= tf.keras.initializers.glorot_normal(seed=42))(dense_layer1)
              

model_char =Model(inputs=layer_0,outputs=output_layer) 

In [62]:
# model summary

model_char.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 5000)]            0         
                                                                 
 embedding_1 (Embedding)     (None, 5000, 41)          1681      
                                                                 
 conv1d_7 (Conv1D)           (None, 4998, 64)          7936      
                                                                 
 conv1d_8 (Conv1D)           (None, 4996, 64)          12352     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 999, 64)          0         
 1D)                                                             
                                                                 
 conv1d_9 (Conv1D)           (None, 997, 64)           12352     
                                                           

In [64]:
# compiling and training model

from tensorflow_addons.metrics import F1Score

checkpoint = ModelCheckpoint(filepath='best_model_2.h5',verbose=1,monitor='val_accuracy',
                           mode='auto',save_best_only=True)

## Tensorboard
log_dir = "logs"
tensorboard = TensorBoard(log_dir=log_dir,histogram_freq=1,write_graph=True)

## compile model 
model_char.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy',F1Score(average='micro',num_classes=20)])



## Trainning 


model_char.fit(x_train,y_train,epochs=10,verbose=2,validation_data=(x_test,y_test),batch_size =64,callbacks=[checkpoint,tensorboard]) 

Epoch 1/10

Epoch 1: val_accuracy improved from -inf to 0.05250, saving model to best_model_2.h5
221/221 - 134s - loss: 3.0631 - accuracy: 0.0504 - f1_score: 0.0504 - val_loss: 3.0637 - val_accuracy: 0.0525 - val_f1_score: 0.0525 - 134s/epoch - 609ms/step
Epoch 2/10

Epoch 2: val_accuracy improved from 0.05250 to 0.05271, saving model to best_model_2.h5
221/221 - 135s - loss: 3.0523 - accuracy: 0.0493 - f1_score: 0.0493 - val_loss: 3.0487 - val_accuracy: 0.0527 - val_f1_score: 0.0527 - 135s/epoch - 612ms/step
Epoch 3/10

Epoch 3: val_accuracy improved from 0.05271 to 0.05292, saving model to best_model_2.h5
221/221 - 142s - loss: 3.0523 - accuracy: 0.0499 - f1_score: 0.0499 - val_loss: 3.0491 - val_accuracy: 0.0529 - val_f1_score: 0.0529 - 142s/epoch - 642ms/step
Epoch 4/10

Epoch 4: val_accuracy did not improve from 0.05292
221/221 - 143s - loss: 3.0526 - accuracy: 0.0467 - f1_score: 0.0467 - val_loss: 3.0640 - val_accuracy: 0.0529 - val_f1_score: 0.0529 - 143s/epoch - 645ms/step
Epoc

### Model-1: Using 1D convolutions with word embeddings

<pre>
<b>Encoding of the Text </b> --> For a given text data create a Matrix with Embedding layer as shown Below. 
In the example we have considered d = 5, but in this assignment we will get d = dimension of Word vectors we are using.
 i.e if we have maximum of 350 words in a sentence and embedding of 300 dim word vector, 
 we result in 350*300 dimensional matrix for each sentance as output after embedding layer
<img src='https://i.imgur.com/kiVQuk1.png'>
Ref: https://i.imgur.com/kiVQuk1.png

<b>Reference:</b>
<a href='https://stackoverflow.com/a/43399308/4084039'>https://stackoverflow.com/a/43399308/4084039</a>
<a href='https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/'>https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/</a>

<b><a href='https://stats.stackexchange.com/questions/270546/how-does-keras-embedding-layer-work'>How EMBEDDING LAYER WORKS </a></b>

</pre>

### Go through this blog, if you have any doubt on using predefined Embedding values in Embedding layer - https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/

<img src='https://i.imgur.com/fv1GvFJ.png'>
ref: 'https://i.imgur.com/fv1GvFJ.png'

<pre>
1. all are Conv1D layers with any number of filter and filter sizes, there is no restriction on this.

2. use concatenate layer is to concatenate all the filters/channels. 

3. You can use any pool size and stride for maxpooling layer.

4. Don't use more than 16 filters in one Conv layer becuase it will increase the no of params. 
( Only recommendation if you have less computing power )

5. You can use any number of layers after the Flatten Layer.
</pre>

### Model-2 : Using 1D convolutions with character embedding

<pre>
<pre><img src="https://i.ytimg.com/vi/CNY8VjJt-iQ/maxresdefault.jpg" width="70%">
Here are the some papers based on Char-CNN
 1. Xiang Zhang, Junbo Zhao, Yann LeCun. <a href="http://arxiv.org/abs/1509.01626">Character-level Convolutional Networks for Text Classification</a>.NIPS 2015
 2. Yoon Kim, Yacine Jernite, David Sontag, Alexander M. Rush. <a href="https://arxiv.org/abs/1508.06615">Character-Aware Neural Language Models</a>. AAAI 2016
 3. Shaojie Bai, J. Zico Kolter, Vladlen Koltun. <a href="https://arxiv.org/pdf/1803.01271.pdf">An Empirical Evaluation of Generic Convolutional and Recurrent Networks for Sequence Modeling</a>
 4. Use the pratrained char embeddings <a href='https://github.com/minimaxir/char-embeddings/blob/master/glove.840B.300d-char.txt'>https://github.com/minimaxir/char-embeddings/blob/master/glove.840B.300d-char.txt</a>
</pre>

<img src='https://i.imgur.com/EuuoJtr.png'>